Phrase based filtering results. Considers impact on set based measures and ranking.

In [ ]:
# looking at phrase boolean queries

%matplotlib inline

import sys 
import os 

nb_dir = os.getcwd()
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from plotlib.loaders import *
from plotlib.plotters import *

from phdconf import config
from phdconf.config import * 

import copy

In [ ]:
queries = load_queries(config.AUS_TOPIC_PATH)
broad, specific = load_query_types(queries)
law, fact, generic = load_query_focus_types(queries)

In [ ]:
index_names = ['filtered-phrasestop']#, config.SIGIR_INDEX_NAME]
qrel_paths = [config.AUS_QREL_PATH]#, config.SIGIR_QREL_PATH]
rel_levels = [config.AUS_REL_LEVEL]#, config.SIGIR_REL_LEVEL]
display_names = ['AUS']#, 'SIGIR']

In [ ]:
base_df = load_1d_dfs(index_names, qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1)[0][0]
base_qry = load_1d_dfs(index_names, qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1, per_query=True)[0][0]

In [ ]:
base_qry_set = load_1d_dfs(index_names, qrel_paths, os.path.join(BASE_DIR, 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1, per_query=True)[0][0]

In [ ]:
def res_per_qry(path:str):
    res = {}
    with open(path) as f:
        for line in f:
            parts = line.split()
            if parts[0] not in res:
                res[parts[0]] = 0
                
            res[parts[0]] += 1
            
    return res

set_res = res_per_qry(os.path.join(BASE_DIR, 'dp-set', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'))

# Average number of documents

In [ ]:
np.asarray(list(set_res.values())).mean()

In [ ]:
np.asarray(list(res_per_qry(os.path.join(BASE_DIR, 'phrases', 'phrase-all-or.run')).values())).mean()

In [ ]:
small_queries = [7, 24, 57, 70, 76, 85, 96]

# Phrase results

In [ ]:
runs = ['phrase-all-or.run', 'phrase-all-and.run', 'phrase-or.run', 'phrase-and.run']
names = ['all-or', 'all-and', 'or', 'and']

In [ ]:
all_qry = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, os.path.join(BASE_DIR, 'phrases'), [runs[0]], per_query=True)

In [ ]:
diff = all_qry[0]-base_qry
metrics = copy.copy(config.METRIC_NAMES)
del metrics['unjudged@20']
fig = diff[metrics].rename(metrics, axis='columns').plot.box(fontsize=15, boxprops=dict(linestyle='-', linewidth=2), color=dict(boxes='black', whiskers='black', caps='r'), medianprops=dict(linestyle='-', linewidth=2, color='b'), figsize=(16, 4)).axhline(y=0, xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='grey')

In [ ]:
# fig.get_figure().savefig('figures/ausnl-all-or-phrase-qry-comp.pdf')

In [ ]:
diff2 = all_qry[0]-base_qry
diff2 = diff2[diff2.index.isin(small_queries)]
fig = diff2[metrics].rename(metrics, axis='columns').plot.box(fontsize=15, boxprops=dict(linestyle='-', linewidth=2), color=dict(boxes='black', whiskers='black', caps='r'), medianprops=dict(linestyle='-', linewidth=2, color='b'), figsize=(16, 4)).axhline(y=0, xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='grey')

In [ ]:
diff2[metrics]

In [ ]:
base_qry[base_qry.index.isin(small_queries)][metrics]

In [ ]:
# write_table('tables/ausnl-phrase-qry-diff', diff2[metrics].rename(columns=metrics).round(4).to_latex(escape=False))

In [ ]:
# fig.get_figure().savefig('figures/ausnl-all-or-phrase-qry-comp-small-queries.pdf')

In [ ]:
dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, os.path.join(BASE_DIR, 'phrases'), runs)

In [ ]:
df = pd.DataFrame(dfs+[base_df], index=names+['base'])

In [ ]:
ev_files = [('$R$', os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), None)]
ev_files.append(('$R_{all}$', os.path.join(BASE_DIR, 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), None))
for i, r in enumerate(runs):
    ev_files.append((names[i], os.path.join(BASE_DIR, 'phrases', r), None))

metrics = copy.copy(config.METRIC_NAMES)
metrics['set_P'] = 'P'
metrics['set_recall'] = 'R'
metrics['set_F_.5'] = 'F.5'

In [ ]:
df = compute_stat_sig(ev_files, config.AUS_QREL_PATH, ['$R$', '$R_{all}$'], metrics, '1')

In [ ]:
df[metrics]

In [ ]:
# write_table('tables/ausnl-phrase-filter', bold_max(df[metrics]).rename(columns=metrics).round(4).to_latex(escape=False))

In [ ]:
inter = Interpolater(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), normalize=True)

vals = [0, 300, 1050, 3000]
dfs = []

for v in vals:
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join(BASE_DIR, 'phrases', 'case-topics-phrase-scoring-mu-{0:.2f}.run'.format(v)), _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'])[0])
    dfs.append(interped_dfs)

In [ ]:
to = 50
metrics = copy.copy(config.METRIC_NAMES)
del metrics['recall_100']

phrase_interp_fig = plot_tune_1d_comp(['base', 'mle', 'mu=300', 'mu=1050', 'mu=3000'], RERANK_METRICS, [[base_df for x in range(to+1)]] + [x[:to+1] for x in dfs], 0.00, (to)/100, 0.01, legend_x=0.965, ylims=RERANK_YLIMS, styles=['--'])

In [ ]:
# modified so does stat sig to not 0 
def select_1d_max_with_interp(display_names, dfs, start, increment, name, interp, base_qry, base_df, base_val, path, qrel_path, rel_level, metrics=None):
    measure_max = {}
    for i in range(len(display_names)):
        for j in range(1, len(dfs[i])):
            for m in dfs[i][j].index:
                if m not in metrics: 
                    continue 
                val = dfs[i][j][m]
                if (display_names[i], metrics[m]) not in measure_max: 
                    measure_max[(display_names[i], metrics[m])] = {'-': val, name: '{0:.2f}'.format(j*increment+start)}
                else: 
                    if measure_max[(display_names[i], metrics[m])]['-'] < val:
                        measure_max[(display_names[i], metrics[m])] = {'-': val, name: '{0:.2f}'.format(j*increment+start)}

    back_metric = {v: k for k, v in metrics.items()}
    for k, v in measure_max.items():
        if k[1] == 'Unjudged@20':
            continue
        _l = float(v[name])
        if _l == 0.00:
            v['-'] = '{0:.4f}'.format(v['-'])
        else:
            print(path, k[0])
            interp.interpolate(path.format(k[0]), _l, 'tmp.run')
            comp = load_dfs(qrel_path, rel_level, '', ['tmp.run'], per_query=True)[0]
            p = stats.ttest_rel(base_qry[back_metric[k[1]]], comp[back_metric[k[1]]]).pvalue
            if p < 0.01:
                v['-'] = '{0:.4f}'.format(v['-'])+'$^{**}$'
            elif p < 0.05:
                v['-'] = '{0:.4f}'.format(v['-'])+'$^{*}$'
            else:
                v['-'] = '{0:.4f}'.format(v['-'])
                
        
    for x in base_df.items():
        if x[0] not in metrics: 
            continue
        measure_max[('base', metrics[x[0]])] = {'-': '{0:.4f}'.format(x[1]), name: '{0:.2f}'.format(base_val)}
        
    max_df = pd.DataFrame.from_dict(measure_max).stack().unstack(level=0)
    return max_df.reindex(list(metrics.values()))

In [ ]:
phrase_interp_max = select_1d_max_with_interp([300, 1050, 3000], dfs, 0.0, 0.01, '$\lambda$', inter, base_qry, base_df, 1050, os.path.join(BASE_DIR,  'phrases', 'case-topics-phrase-scoring-mu-{0:.2f}.run'), config.AUS_QREL_PATH, config.AUS_REL_LEVEL, metrics=metrics).T

In [ ]:
phrase_interp_max